In [1]:
%%capture
%run utils.ipynb
%run quadrotor_model.ipynb

# Sensors

This notebook implements the sensors that are used on-board a quadcopter: accelerometer, gyro, GPS, camera, etc. Each sensor implemented herein inherits from the following abstract `Sensor` class and must override the `read` method.

In [2]:
class Sensor(object):
    """Sensor
    
    An abstract base class for all sensors.
    """
    def __init__(self):
        self.name = "Abstract Sensor"
        
    def __str__(self):
        return self.name
    
    def read(self, quad, n, Ts):
        return 0

## Sensor Manager

In order to allow flexibility in sensor configurations, a `SensorManager` class is created. A custom set of sensors is registered with the manager for each simulation. During a simulation, the [quadsim](quadsim.ipynb) `Simulator` class asks the sensor manager to produce a data packet that represents the current sensor readings from the suite of on-board sensors.

In [3]:
class SensorManager(object):
    """Sensor Manager
    """
    def __init__(self):
        # create a list for sensor objects
        self.sensors = []
    
    def register(self, sensor):
        self.sensors += [sensor]
    
    def get_data_packet(self, quad, i, Ts):
        # dictionary of sensor data, keyed by sensor name
        pkt = {}
        
        for s in self.sensors:
            pkt[s.name] = s.read(quad, i, Ts)
            
        return pkt

## Camera

Electro-optical (EO) cameras are extremely useful in autonomy and robotics. Their rich source of visual information enables a wide variety of applications. Robotic vision is a very active research area, with common themes such as: vision-based simultaneous localization and mapping (SLAM), visual-inertial odometry, object recognition with convolutional neural networks, image-based visual servoing, and target tracking.

In this simulation, the purpose of the camera is to measure normalized bearing vectors to interesting points such as pixel features, targets, or landmarks. As such, we will focus solely on aspects of camera geometry (as opposed to semantic understanding, pixel intensities, etc). Feature locations in the inertial frame are given to the camera which then projects them onto the pixel plane.

### Coordinate Frames
There are a number of important coordinate frames associated with a camera sensor.

### Camera Geometry

The pinhole camera model is the most commonly used camera model. This allows us to simplify the optical characteristics and focus on the geometry of how 3D objects are imaged. Suppose that the point $P$ exists in 3D space and can be expressed in the inertial frame as $P^i = \begin{bmatrix} x^i & y^i & z^i\end{bmatrix}^\top$. Using the pinhole camera model, the perspective projection model is used to image this point as
$$
\begin{equation}
\lambda \begin{bmatrix} u \\ v \\ 1 \end{bmatrix} =
\begin{bmatrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{bmatrix}
\left[ R_i^c \mid t_i^c \right]
\begin{bmatrix} x^i \\ y^i \\ z^i \\ 1 \end{bmatrix},
\end{equation}
$$
where $K$ is the *camera calibration matrix*, *intrinsic camera matrix*, or simply *camera matrix*.

This gives a result in pixels in the image plane attached to the camera frame, see the figure below.


Written out in component form, the perspective projection equations are
$$
\begin{align}
u &= fx' + c_x = f\frac{x}{z} + c_x \\
v &= fy' + c_y = f\frac{y}{z} + c_y
\end{align}
$$

<div style="text-align:center">
<img src="assets/camera_geometry.svg" width="80%"/>
Figure 1: Camera geometry
</div>

In [46]:
class Camera(Sensor):
    """Camera
    """
    def __init__(self, fps=30, size=(800,600), hfovd=33.99, pxnoise=0, tick=None):
        self.name = "Camera"
        
        # Register a callback that gets called when a new
        # frame is to be registered. The user can use this
        # callback to propagate features in the camera's
        # FOV. The function can return inertial positions
        # to the camera to be imaged and projected onto
        # the pixel plane. The function signature is:
        #
        #     def tick_fn(capture, n, Ts)
        #
        self.fns = [tick] if tick else []
        
        # Calculate camera parameters
        self.fps = fps
        self.size = size
        self.hfov = np.radians(hfovd) # angular FOV for width (degrees)
        self.f_px = f = (size[0]/2)/np.tan(self.hfov/2)
        self.pxnoise_std = pxnoise
        cx = size[0]/2
        cy = size[1]/2
        self.K = np.array([[f, 0, cx],
                           [0, f, cy],
                           [0, 0, 1]])
        
        # Camera transformation from quadrotor body frame
        self.t = np.zeros((3,1))
        self.R = np.eye(3)
        
        # Last camera measurement
        self.meas = None
        
        # how many sensor reads have there been
        self.ticks = 0
        
    def _camera_projection(self, quad, inertial):
        """Camera Projection
        
        Uses the camera matrix to project features onto the pixel plane.
        """
        
        # given inertial measurements of features, calculate
        # the feature positions in the vehicle frame
        p_veh = inertial - quad.r
        
        # transform to the camera frame
        # TODO: Add in translation from body to camera
        Rv2b = Rot_v_to_b(*quad.Phi.flatten())
        Rb2c = self.R
        p_cam = Rb2c.dot(Rv2b.dot(p_veh))
        
        # perspective transformation using camera matrix
        # force homogeneous coordinates (normalize out depth)
        hpx = self.K.dot(p_cam)
        hpx = hpx / hpx[2,:]
        
        # extract pixels from homogeneous coordiantes
        # and add pixel noise (AWGN)
        px = hpx[:2,:]
        px += self.pxnoise_std*np.random.randn(*px.shape)
        
        # TODO: remove measurements outside of FOV
        
        return px        
    
    def set_transformation(self, t=None, R=None):
        """Set Transformation
        
        Sets the transformation from the quadrotor body frame
        to the camera frame. The transformation is represented
        as a translation t and rotation R.
        """
        self.t = t if t is not None else np.zeros((3,1))
        self.R = R if R is not None else np.eye(3)
        
    def register_feature(self, feature):
        self.fns += [feature.tick]        
    
    def read(self, quad, n, Ts):
        
        # empty list of measurements to be filled by tick functions
        inertial_measurements = None
        
        # Call any associated tick functions
        capture = True
        for fn in self.fns:
            data = fn(capture, n, Ts)
            if data is not None:
                if inertial_measurements is None:
                    inertial_measurements = data
                else:
                    inertial_measurements = np.hstack((inertial_measurements, data))
        
        # if no measurements were received, bail now
        if inertial_measurements is None:
            return None
        
        # Enforce fps (to closest Ts) before doing camera stuff
        if np.mod(self.ticks,round(1/(self.fps*Ts))) == 0:
            # project inertial measurements onto pixel plane
            self.meas = self._camera_projection(quad, inertial_measurements)
        
        self.ticks += 1
        
        return self.meas

In [51]:
def target_tick(capture, n, Ts):
    
    # ellipse
    f = 0.01 # Hz
    x = 10*np.sin(2*np.pi*f*n*Ts)
    y = 5*np.cos(2*np.pi*f*n*Ts)
    
    return np.array([[x,y,0]]).T

camera = Camera(tick=target_tick)
camera.set_transformation(R=rot3d(45,0,90))

# Instantiate a quadrotor model with the given initial conditions
quad = Quadrotor(r=np.array([[-30],[0],[-30]]),
                 v=np.array([[0],[0],[0]]),
               Phi=np.array([[0],[0],[0]]))

# How many iterations are needed
Ts = 0.01
Tf = 0.1
N = int(Tf/Ts)

for i in range(N):
    x = camera.read(quad, i, Ts)
    print(x.T)

[[ 554.238  300.000]]
[[ 554.238  300.000]]
[[ 554.238  300.000]]
[[ 554.189  299.589]]
[[ 554.189  299.589]]
[[ 554.189  299.589]]
[[ 554.140  299.178]]
[[ 554.140  299.178]]
[[ 554.140  299.178]]
[[ 554.090  298.768]]


## Accelerometer

## Rate Gyro

## GPS